In [1]:
import os
import cPickle as pickle
import numpy as np
from nn1_decouple_feat import *
print "start"

start


In [2]:
from YellowFin_Pytorch.tuner_utils.yellowfin import YFOptimizer

In [3]:
os.environ['CUDA_VISIBLE_DEVICES']="1"
batch_size = 250
num_classes = 1

In [4]:
data = pickle.load(open("yf_data.dat", "rb"))
X_train = data['X_train']
X_train_features = data['X_train_features']
Y_marginals = data['Y_marginals']

X_test = data['X_test']
X_test_feature = data['X_test_feature']

In [5]:
print X_train_features.shape[1], X_test_feature.shape[0]

10344 6714


In [6]:
word_attn = AttentionWordRNN(batch_size=batch_size, num_tokens=5664, embed_size=100, word_gru_hidden=50, bidirectional= True)
mix_softmax = MixtureSoftmax(batch_size=batch_size, word_gru_hidden = 50, feature_dim = X_train_features.shape[1], n_classes=num_classes)
# mix_softmax = MixtureSoftmax(batch_size=batch_size, word_gru_hidden = 50, feature_dim = 0, n_classes=num_classes)

In [7]:
softmax = nn.Softmax()
sigmoid = nn.Sigmoid()

In [8]:
learning_rate = 0.1

optimizer = YFOptimizer(mix_softmax.parameters(), beta=0.999, lr=learning_rate, mu=0.0, zero_debias=True, clip_thresh=None, auto_clip_fac=None, curv_win_width=20)

# word_optmizer = YFOptimizer(word_attn.parameters(), lr=learning_rate, mu=0.0, auto_clip_fac=2.0)
# mix_optimizer = YFOptimizer(mix_softmax.parameters(), lr=learning_rate, mu=0.0, auto_clip_fac=2.0)

criterion = nn.MultiLabelSoftMarginLoss(size_average=True)

In [9]:
word_attn.cuda()
mix_softmax.cuda()

MixtureSoftmax (
  (linear): Linear (10344 -> 1)
)

In [10]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [11]:
def plot_func(log_dir, iter_id, loss_list, local_curv_list, max_curv_list, min_curv_list,
             lr_g_norm_list, lr_list, dr_list, mu_list, grad_avg_norm_list,
             dist_list, grad_var_list):
    def running_mean(x, N):
        cumsum = np.cumsum(np.insert(x, 0, 0)) 
        return (cumsum[N:] - cumsum[:-N]) / N 
    plt.figure()
    plt.semilogy(loss_list, '.', alpha=0.2, label="Loss")
    plt.semilogy(running_mean(loss_list,100), label="Average Loss")
    plt.xlabel('Iterations')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()
    ax = plt.subplot(111)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.05),
          ncol=3, fancybox=True, shadow=True)
    plt.savefig(log_dir + "/fig_loss_iter_" + str(iter_id) + ".pdf")
    plt.close()

    plt.figure()
    plt.semilogy(local_curv_list, label="local curvature")
    plt.semilogy(max_curv_list, label="max curv in win")
    plt.semilogy(min_curv_list, label="min curv in win")
#         plt.semilogy(clip_norm_base_list, label="Clipping Thresh.")
    plt.semilogy(lr_g_norm_list, label="lr * grad norm")
    plt.title("On local curvature")
    plt.grid()
    ax = plt.subplot(111)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.05),
          ncol=2, fancybox=True, shadow=True)
    plt.savefig(log_dir + "/fig_curv_iter_" + str(iter_id) + ".pdf")
    plt.close()

    plt.figure()
    plt.semilogy(lr_list, label="lr min")
    plt.semilogy(dr_list, label="dynamic range")
    plt.semilogy(mu_list, label="mu")
    plt.semilogy(grad_avg_norm_list, label="Grad avg norm")
    plt.semilogy(dist_list, label="Est dist from opt")
    plt.semilogy(grad_var_list, label="Grad variance")
    plt.title('LR='+str(lr_list[-1])+' mu='+str(mu_list[-1] ) )
    plt.grid()
    plt.legend(loc="upper right")
    plt.savefig(log_dir + "/fig_hyper_iter_" + str(iter_id) + ".pdf")
    plt.close()

In [ ]:
def train_early_stopping(mini_batch_size, X_train, X_train_feature, y_train, X_test, X_test_feature, word_attn_model, sent_attn_model, 
                         optimizer, loss_criterion, num_epoch, 
                         print_val_loss_every = 1000, print_loss_every = 50, print_figure_every=2500):
    start = time.time()
    loss_full = []
    loss_epoch = []
    accuracy_epoch = []
    loss_smooth = []
    accuracy_full = []
    epoch_counter = 0
    print "mini_batch_size", mini_batch_size
    g = gen_minibatch(X_train, X_train_feature, y_train,  mini_batch_size)
    
    # DEBUG
    loss_list = []
    h_max_list = []
    h_min_list = []
    h_list = []
    dist_list = []
    grad_var_list = []
    
    lr_g_norm_list = []
    
    lr_list = []
    dr_list = []
    mu_list = []
    grad_avg_norm_list = []
    
    plot_figure = plt.figure()
    # END of DEBUG
    
    for i in xrange(1, num_epoch + 1):
#         print 'epoch ', i, timeSince(start)

        # DEBUG
#         if i < 2000:
#             optimizer._lr = 0.1
#         optimizer._mu = 0.98
        # END of DEBUG

        try:
            tokens, features, labels = next(g)
#             print labels
#             print 'tokens', tokens
            loss = train_data(tokens, features, labels, word_attn_model, sent_attn_model, optimizer, loss_criterion)
#             print loss
            acc = test_accuracy_mini_batch(tokens, features, labels, word_attn_model, sent_attn_model)
            accuracy_full.append(acc)
            accuracy_epoch.append(acc)
            loss_full.append(loss)
            loss_epoch.append(loss)
            
#             print "optimizer", optimizer._lr, optimizer._mu, optimizer._h_min, optimizer._h_max, optimizer._dist_to_opt, optimizer._grad_var, loss
#             print loss every n passes
            if i % print_loss_every == 0:
                print 'Loss at %d minibatches, %d epoch,(%s) is %f' %(i, epoch_counter, timeSince(start), np.mean(loss_epoch))
                print 'Accuracy at %d minibatches is %f' % (i, np.mean(accuracy_epoch))
        except StopIteration:
            epoch_counter += 1
            print 'Reached %d epocs' % epoch_counter
            print 'i %d' % i
            print 'loss_epoch', np.sum(loss_epoch) / X_train.shape[0]
            print "optimizer", optimizer._lr, optimizer._mu, optimizer._h_min, optimizer._h_max, optimizer._dist_to_opt, optimizer._grad_var

#             print "word_optimizer", word_optmizer._lr, word_optmizer._mu, word_optmizer._h_min, word_optmizer._h_max, word_optmizer._dist_to_opt, word_optmizer._grad_var
#             print "mix_optimizer", mix_optimizer._lr, mix_optimizer._mu, mix_optimizer._h_min, mix_optimizer._h_max, mix_optimizer._dist_to_opt, mix_optimizer._grad_var
#             print loss_epoch
            p = test_accuracy_full_batch(X_test, X_test_feature, mini_batch_size, word_attn_model, sent_attn_model)
#             print p
#             print len(p)
#             p = np.ravel(p)
#             pos = []
#             for i, candidate in enumerate(test_candidates):
#                 test_label_index = L_test.get_row_index(candidate)
#                 test_label       = L_test[test_label_index, 0]
#                 if i < len(p) and p[i] > 0.5:
#                     pos.append(candidate)
                
#             (TP, FP, FN) = entity_level_f1(pos, gold_file, ATTRIBUTE, corpus, parts_by_doc_test)
                
            g = gen_minibatch(X_train, X_train_feature, y_train, mini_batch_size)
            loss_epoch = []
            accuracy_epoch = []
            
        # DEBUG
        loss_list.append(loss)
        h_max_list.append(optimizer._h_max)
        h_min_list.append(optimizer._h_min)
        h_list.append(optimizer._global_state['grad_norm_squared'] )
        
#         print "test in the middle ", h_list[-1], h_max_list[-1], h_min_list[-1]
#         print "curv window ", optimizer._global_state["curv_win"]
        
        dist_list.append(optimizer._dist_to_opt)
        grad_var_list.append(optimizer._grad_var)

        lr_g_norm_list.append(optimizer._lr * np.sqrt(optimizer._global_state['grad_norm_squared'] ) )

        lr_list.append(optimizer._lr)
        dr_list.append(optimizer._h_max / optimizer._h_min)
        mu_list.append(optimizer._mu)
        grad_avg_norm_list = []
#         if (i % 1000 == 0) and i != 0:
#             with open("h_val.txt", "w") as f:
#                 np.savetxt(f, h_list)
        
        if (i % print_figure_every == 0 and i != 0) or (i == 50 or i == 1000):
            plot_func(log_dir=log_dir, iter_id=i, loss_list=loss_list, 
                 local_curv_list=h_list, max_curv_list=h_max_list, 
                 min_curv_list=h_min_list, lr_g_norm_list=lr_g_norm_list, 
                 lr_list=lr_list, dr_list=dr_list, mu_list=mu_list, 
                 grad_avg_norm_list=grad_avg_norm_list,
                 dist_list=dist_list, grad_var_list=grad_var_list)
            print "figure plotted"
        # END of DEBUG
        
#     torch.save(word_attn_model, log_dir + "/word_attn.model")
#     torch.save(sent_attn_model, log_dir + "/sent_attn.model")
            
    return loss_full

In [ ]:
log_dir = "./YF_feat_only-0.1-init-win_20_log_smooth_debias-batch-250-norm-of-average-grad-refine-non-zero"
if not os.path.isdir(log_dir):
    os.mkdir(log_dir)
loss_full= train_early_stopping(batch_size, X_train, X_train_features, Y_marginals, X_test, X_test_feature, word_attn, mix_softmax, optimizer, 
                            criterion, 100000, 1000, 1000)

mini_batch_size 250
figure plotted
Reached 1 epocs
i 53
loss_epoch 0.000875870376198
optimizer 0.11191462285 0.0384687598543 0.00848387882162 0.147495418749 0.84029298961 0.110313765705
(6714, 2)
0
Reached 2 epocs
i 106
loss_epoch 0.000786514649855
optimizer 0.132035449253 0.0821332376003 0.00360401772717 0.0230364807379 0.942392598112 0.0571836531162
(6714, 2)
0
Reached 3 epocs
i 159
loss_epoch 0.000768780948506
optimizer 0.158559935348 0.12445213905 0.00221322121678 0.0112655467199 1.01959157986 0.0382137149572
(6714, 2)
0
Reached 4 epocs
i 212
loss_epoch 0.000757431808556
optimizer 0.190784638387 0.165012186135 0.00160530114257 0.00741620492775 1.084024623 0.028447566554
(6714, 2)
0
Reached 5 epocs
i 265
loss_epoch 0.000749276893335
optimizer 0.227834153852 0.203791632255 0.00119973746109 0.0054270426627 1.13943613733 0.0224720295519
(6714, 2)
0
Reached 6 epocs
i 318
loss_epoch 0.000743209690181
optimizer 0.268638617477 0.240872574283 0.000929781947 0.00427797564967 1.18742398587 0.

Reached 49 epocs
i 2597
loss_epoch 0.000721857665138
optimizer 1.13406773404 0.890800648432 0.000483383584802 0.01108003955 0.661407587535 0.0236846525222
(6714, 2)
0
Reached 50 epocs
i 2650
loss_epoch 0.000721031738714
optimizer 1.09335092617 0.895096563978 0.000438058176976 0.01000862665 0.628226336901 0.0224250257015
(6714, 2)
0
Reached 51 epocs
i 2703
loss_epoch 0.000720650916384
optimizer 1.05369020378 0.899265802385 0.000395415952468 0.00920878307077 0.597026809867 0.02124113217
(6714, 2)
0
Reached 52 epocs
i 2756
loss_epoch 0.000721048953905
optimizer 1.01503999031 0.903316958667 0.00036040178626 0.0083810754271 0.567685885118 0.0201184488833
(6714, 2)
0
Reached 53 epocs
i 2809
loss_epoch 0.000726591254738
optimizer 0.977657729394 0.907211793318 0.000361020430028 0.00823125005883 0.540080772193 0.0192273315042
(6714, 2)
0
Reached 54 epocs
i 2862
loss_epoch 0.000724061949483
optimizer 0.941780729657 0.910911432508 0.000365904777989 0.00816222052178 0.514079345949 0.0182768069208


Reached 96 epocs
i 5088
loss_epoch 0.00071979225013
optimizer 0.219238016906 0.983683525664 7.16982905905e-05 0.000894991734578 0.169486352108 0.00223909132183
(6714, 2)
0
Reached 97 epocs
i 5141
loss_epoch 0.000719732073111
optimizer 0.215498975776 0.984186367171 6.9686634241e-05 0.000854458022305 0.170734036169 0.00213112356141
(6714, 2)
0
Reached 98 epocs
i 5194
loss_epoch 0.000720282431795
optimizer 0.212226927372 0.984661084084 6.9104429012e-05 0.00081472022231 0.172289098242 0.00203004595824
(6714, 2)
0
Reached 99 epocs
i 5247
loss_epoch 0.000719932505432
optimizer 0.209442389806 0.985106773954 6.79394943344e-05 0.000793599939896 0.174144015234 0.0019377854187
(6714, 2)
0
Reached 100 epocs
i 5300
loss_epoch 0.000720045280617
optimizer 0.207147824585 0.985524093446 6.78071352836e-05 0.000788926163803 0.176308099725 0.00185316777788
(6714, 2)
0
Reached 101 epocs
i 5353
loss_epoch 0.000719859919125
optimizer 0.205343552029 0.985914792884 6.68862193991e-05 0.000772003528127 0.1787506

Reached 143 epocs
i 7579
loss_epoch 0.000762617787761
optimizer 0.249094607957 0.983378635307 0.00089843299035 0.0104252678921 0.160473975955 0.00787474773824
(6714, 2)
0
Reached 144 epocs
i 7632
loss_epoch 0.000776984764966
optimizer 0.243052813721 0.983109268582 0.000969487761173 0.0111765830895 0.155224617571 0.00822582002729
(6714, 2)
0
Reached 145 epocs
i 7685
loss_epoch 0.000758215574474
optimizer 0.237079622422 0.982839351076 0.0010176548182 0.0114372640475 0.149873779125 0.00810038391501
(6714, 2)
0
Reached 146 epocs
i 7738
loss_epoch 0.000744581777533
optimizer 0.231270571319 0.982575968954 0.00104440719406 0.0115101794915 0.14464362655 0.00793472211808
(6714, 2)
0
Reached 147 epocs
i 7791
loss_epoch 0.000734329068274
optimizer 0.225601141405 0.982334000976 0.00104237163439 0.0111106408242 0.139522923511 0.00766756618395
(6714, 2)
0
Reached 148 epocs
i 7844
loss_epoch 0.000728432696472
optimizer 0.220054785837 0.982126470033 0.00100811750863 0.0105264352801 0.134608240024 0.00

figure plotted
Reached 189 epocs
i 10017
loss_epoch 0.000719068468054
optimizer 0.10836366096 0.991907821342 5.1207794902e-05 0.000596825369915 0.130602691988 0.00107463635504
(6714, 2)
0
Reached 190 epocs
i 10070
loss_epoch 0.00071954879261
optimizer 0.110591123081 0.992037937354 4.94777313365e-05 0.000566107671786 0.134851371401 0.00102489918936
(6714, 2)
0
Reached 191 epocs
i 10123
loss_epoch 0.000719369342931
optimizer 0.113216138684 0.992156647575 4.78176573585e-05 0.000548552431877 0.139362416932 0.00097933143843
(6714, 2)
0
Reached 192 epocs
i 10176
loss_epoch 0.000719278983369
optimizer 0.116231711976 0.992265978678 4.61117510928e-05 0.000540433011095 0.14419013733 0.00093795650173
(6714, 2)
0
Reached 193 epocs
i 10229
loss_epoch 0.000719510394446
optimizer 0.119708241994 0.992362106702 4.65270277527e-05 0.000546042296679 0.149303202721 0.000903131789528
(6714, 2)
0
Reached 194 epocs
i 10282
loss_epoch 0.000720165937197
optimizer 0.123749624253 0.992439407741 4.74802339443e-05 

Reached 235 epocs
i 12455
loss_epoch 0.000725841382598
optimizer 0.173887729529 0.985188368303 0.00095360527757 0.0107309938765 0.114385223342 0.00900434888899
(6714, 2)
0
figure plotted
Reached 236 epocs
i 12508
loss_epoch 0.00072253194907
optimizer 0.168991899116 0.985086403153 0.000887178357948 0.00988396327912 0.109839103327 0.0085868909955
(6714, 2)
0
Reached 237 epocs
i 12561
loss_epoch 0.000720723069322
optimizer 0.164141899397 0.98505118861 0.000794570749302 0.00870210517569 0.105508880242 0.00816509313881
(6714, 2)
0
Reached 238 epocs
i 12614
loss_epoch 0.000720164808801
optimizer 0.15934146285 0.985078205507 0.00071388104355 0.00751568514377 0.101420748943 0.00776409264654
(6714, 2)
0
Reached 239 epocs
i 12667
loss_epoch 0.000720038632894
optimizer 0.154590523315 0.985164085526 0.000630865313101 0.00658540011627 0.0975900716138 0.0073812850751
(6714, 2)
0
Reached 240 epocs
i 12720
loss_epoch 0.000719659936527
optimizer 0.14989008621 0.985304782112 0.000558976074128 0.00595890

Reached 281 epocs
i 14893
loss_epoch 0.000719538777075
optimizer 0.0705459199627 0.994831315065 3.57951266096e-05 0.000360320596174 0.10507811697 0.000931738410145
(6714, 2)
0
Reached 282 epocs
i 14946
loss_epoch 0.000719300370761
optimizer 0.072657338559 0.994890711573 3.55302158457e-05 0.000353406144289 0.109069847113 0.000890511961188
(6714, 2)
0
Reached 283 epocs
i 14999
loss_epoch 0.000719857902527
optimizer 0.075126989187 0.994939167788 3.57126414693e-05 0.000356578256606 0.113244619652 0.000852663186379
(6714, 2)
0
Loss at 15000 minibatches, 283 epoch,(53m 26s) is 0.191853
Accuracy at 15000 minibatches is 1.000000
figure plotted
Reached 284 epocs
i 15052
loss_epoch 0.000719645031277
optimizer 0.0779872496393 0.994976209455 3.52727870683e-05 0.000353332323579 0.117562745862 0.000815937004518
(6714, 2)
0
Reached 285 epocs
i 15105
loss_epoch 0.000719689270729
optimizer 0.0811970813935 0.995005832518 3.50821153545e-05 0.000356239561028 0.122101665983 0.000782420684118
(6714, 2)
0
Re

Reached 327 epocs
i 17331
loss_epoch 0.000738205894765
optimizer 0.134613707086 0.986536038062 0.00130087407947 0.0133339270465 0.108458502934 0.011085588485
(6714, 2)
0
Reached 328 epocs
i 17384
loss_epoch 0.000732132499744
optimizer 0.131328930871 0.98625955312 0.001279900054 0.0131470612693 0.104253048085 0.0107246469706
(6714, 2)
0
Reached 329 epocs
i 17437
loss_epoch 0.000729188207524
optimizer 0.128077757038 0.986027347481 0.00123651737689 0.0128872002166 0.100206889033 0.0103634260595
(6714, 2)
0
Reached 330 epocs
i 17490
loss_epoch 0.000725261811342
optimizer 0.124847842128 0.9858464732 0.00117132323375 0.0123991772818 0.096230552061 0.00994761474431
(6714, 2)
0
figure plotted
Reached 331 epocs
i 17543
loss_epoch 0.00072333826144
optimizer 0.121621084273 0.985727583315 0.0010697956687 0.0119619149227 0.0924229937549 0.00956237036735
(6714, 2)
0
Reached 332 epocs
i 17596
loss_epoch 0.000721457737759
optimizer 0.118381919974 0.985677505037 0.000960059833387 0.0112598239647 0.0887

Reached 373 epocs
i 19769
loss_epoch 0.000719456279488
optimizer 0.0435184754474 0.995602781361 2.96543140776e-05 0.000315586972072 0.0729629830868 0.00116130791139
(6714, 2)
0
Reached 374 epocs
i 19822
loss_epoch 0.00071934883863
optimizer 0.0441499679984 0.995695153544 2.97427615428e-05 0.000319018721191 0.0757040548086 0.00111156795174
(6714, 2)
0
Reached 375 epocs
i 19875
loss_epoch 0.000719216410552
optimizer 0.0449816174035 0.995777538686 2.95583562977e-05 0.000323230410128 0.0785271536635 0.0010648961179
(6714, 2)
0
Reached 376 epocs
i 19928
loss_epoch 0.000718834126623
optimizer 0.0460080609786 0.995851315331 2.92068344611e-05 0.000323330630846 0.081348235632 0.00101615802851
(6714, 2)
0
Reached 377 epocs
i 19981
loss_epoch 0.000719510341297
optimizer 0.0472292596719 0.995917602503 2.84687250407e-05 0.000324264085345 0.0843597779493 0.00097050529439
(6714, 2)
0
Loss at 20000 minibatches, 377 epoch,(70m 36s) is 0.203636
Accuracy at 20000 minibatches is 1.000000
figure plotted
Re

Reached 418 epocs
i 22154
loss_epoch 0.00098850938819
optimizer 0.250669967588 0.991042716566 0.000413316262568 0.00419508805289 0.190006036999 0.010438430123
(6714, 2)
0
Reached 419 epocs
i 22207
loss_epoch 0.000993587018915
optimizer 0.243229131235 0.990807513602 0.000483866004482 0.00488777469736 0.183766739205 0.0114236017689
(6714, 2)
0
Reached 420 epocs
i 22260
loss_epoch 0.00105648212095
optimizer 0.235792501255 0.990554500843 0.000569181016976 0.00575296957699 0.177779557703 0.0131238214672
(6714, 2)
0
Reached 421 epocs
i 22313
loss_epoch 0.00101970316361
optimizer 0.228535992894 0.990268086041 0.000666016157615 0.00651007622547 0.171719459838 0.0135064991191
(6714, 2)
0
Reached 422 epocs
i 22366
loss_epoch 0.000950022834843
optimizer 0.221551838161 0.98995396779 0.000743689048274 0.00726092354767 0.165638442429 0.0140003822744
(6714, 2)
0
Reached 423 epocs
i 22419
loss_epoch 0.000919196902132
optimizer 0.214755046006 0.989624004153 0.000838521446855 0.00822491964254 0.15971824